In [35]:
%pip install pillow

from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf

# Binary Classification

+ 훈련 데이터를 다운로드 후 디렉터리 구조로 압축을 푸는 코드

In [36]:
import urllib.request
import zipfile

url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url,file_name)

zip_ref = zipfile.ZipFile(file_name,'r')
zip_ref.extractall(training_dir)
zip_ref.close()

+ ImageDataGenerator 객체 train_datagen
+ Iterator 객체 train_generator
+ 이미지가 두 개인 경우이기 때문에 classmode = binary

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size = (300, 300),
    class_mode = 'binary'
)

train_ds = tf.keras.utils.image_dataset_from_directory(
    training_dir,
    image_size=(300, 300),
    label_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 1027 files belonging to 2 classes.


### 말 - 사람 데이터셋을 위한 CNN 구조

+ 이미지가 300 x 300 pixel이기 떄문에 훨씬 많은 층이 필요하다.
+ 흑백이 아니라 컬러 이미지라 채널이 3개가 필요하다.
+ 두 종류의 이미지만 있으므로 하나의 출력 뉴런을 사용하는 이진 분류기를 만들 수 있다. 
+ 이 뉴런은 음성 클래스일 경우 0, 양성 클래스일 경우 1에 가까운 값을 출력

In [38]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(300, 300, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

c:\Users\rlead\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


+ CNN과 Pooling layer를 모두 통과하면 7 X 7 크기가 됨.

In [39]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(learning_rate = 0.001),
              metrics = ['accuracy'])

In [40]:
model.fit(train_ds, epochs = 15)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5925 - loss: 0.7058
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.8054 - loss: 0.4288
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 728ms/step - accuracy: 0.9071 - loss: 0.2532
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 209ms/step - accuracy: 0.9390 - loss: 0.1537
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 196ms/step - accuracy: 0.9667 - loss: 0.0966
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.9559 - loss: 0.1274
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.9824 - loss: 0.0684
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step - accuracy: 0.9919 - loss: 0.0240
Epoch 9/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - accuracy: 0.9775 - loss: 0.1219
Epoch 10/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - accuracy: 0.9954 - loss: 0.0202
Epoch 11/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step - accuracy: 1.0000 - loss: 0.0043
Epoch 12/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 196ms/step - 

### 검증 세트 추가하기



In [41]:
validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"

validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

validation_datagen = ImageDataGenerator(rescale = 1/255)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(300, 300),
    label_mode='binary'
)

Found 256 images belonging to 2 classes.


In [42]:
model.fit(train_generator,
          epochs = 15,
          validation_data = validation_generator)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.